## Boruta (feature selection)

### 변수 선택법의 종류

- Filter Method : 데이터에 대한 통계적 특성을 이용하여 변수를 선택하는 방법
ex) 상호 정보량, 상관계수, AIC, BIC
- Wrapper Method : 모델에서 좋은 성능을 모아서 변수를 선택하는 방법
- Embedded Method : 모델링 기법 자체에 변수 선택이 포함되어 있는 방법

### Boruta 설명

Boruta 알고리즘은 랜덤포레스트를 기반으로 변수를 선택하는 Wrapper Method이다.

기본적인 아이디어는 기존 변수를 복원 추출해서 만든 변수(shadow)보다 모형 생성에 영향을 주지 못했다고 하면 이는 가치가 크지 않은 변수로 인식하여 제거한다.

### Boruta 실행 순서

1. 모든 독립변수에 대한 복제 랜덤 변수를 생성한다.(원 데이터가 5개 적은 경우에도 5개의 변수를 복제한다. 우리는 이러한 복제 변수를 'Shadow features' or 'permuted copies'라고 부른다.

2. 반응변수와의 상관을 제거하기 위해서 복제된 자료를 랜덤하게 섞고 원 자료와 결합한다.

3. 결합된 데이터와 원 데이터에 대해 랜덤포레스트 모형을 생성하여 변수중요도를 계산한다.

4. Z-score를 계산하는데 정확도 손실의 평균을 정확도 손실 표준편차로 나눈 값이다.

$Z = \frac{(x-\mu)}{σ}$

5. shadow 변수들 중에 가장 높은 Z-score(MZSA, Maximum Z score among Shadow Attriburtes)를 찾는다.

6. 원 자료에 대한 Z-score가 MZSA보다 큰 경우 Hit를 +1 해준다.

(Hit +1을 해준다는 의미는 한번 클 떄를 표시해준 것이다.)

7. 위의 6번 과정을 랜덤포레스트가 수행되는 횟수만큼 또는 모든 변수들이 중요한 변수와 중요하지 않은 변수로 tagged 될 때까지 반복한다.

### Tip

feature의 개수가 많다면 너무 많은 시간이 소모된다.

1. GPU를 사용하여 모델을 사용한다면 더 빠르게 진행될 수 있다.
2. Lightgbm을 사용하여 rf를 사용하면 비슷하게 사용할 수 있다.
3. 데이터가 너무 큰 경우에는 샘플을 뽑아서 사용한다면 시간을 좀 더 효율적으로 사용할 수 있을 것 같다.

### 참고자료

https://syj9700.tistory.com/50

### Tutorial

- 데이터양이 많다는 가정하에 0.3씩 sample을 뽑아서 모델을 실행했고 3개의 excel파일을 확인하며 선택된 변수의 교집합, 합집합을 최종 변수로 사용할 수 있습니다.

In [ ]:
# boruta
print('boruta 시작')

a = [1,2,3]
final_green_area = {}

for i in tqdm(range(len(a))):
    print(a[i])
    temp_boruta = ondo_freeze.sample(frac=0.3, random_state=a[i]).reset_index(drop=True)

    # load X and y
    # NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
    x = temp_boruta.drop(['worktime_day'], axis = 1)
    y = temp_boruta['worktime_day']

    # define random forest classifier, with utilising all cores and
    # sampling in proportion to y labels
    rf = RandomForestRegressor(n_jobs=-2, max_depth=1)

    # define Boruta feature selection method
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1, max_iter=2)

    #find all relevant features - 5 features should be selected
    feat_selector.fit(np.array(x), np.array(y))

    ### print results
    green_area = x.columns[feat_selector.support_].to_list()
    blue_area = x.columns[feat_selector.support_weak_].to_list()
    red_area = x.columns.difference(green_area+blue_area)

    dic = { 'Accepted features' : green_area, 'Undecided features' : blue_area, 'Rejected features' : red_area}
    res = pd.DataFrame.from_dict(dic, orient='index')
    res = res.transpose()
    res.to_csv(save_path + 'boruta_' + str(i) + '.xlsx',index=False, encoding = encoding)

print('boruta 끝')